In [7]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import vtk
import os
from scipy.spatial import ConvexHull
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('default')

np.random.seed(23765)

In [8]:
user = 'Kevin'
simulation = 'viz-stimulus'

if user == 'Kattelijn':
    base_folder = f'Users/Kattelijn/Desktop/{simulation}/'
elif user == 'Kevin':
    base_folder = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/{simulation}/'


In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv(os.path.join(base_folder, 'positions/rank_0_positions.txt'), skiprows=8,
                     delimiter=' ', names=column_headers_pos)

,local id,pos x,pos y,pos z,area,type
0,1,88.017654,143.88911,83.259131,area_43,ex
1,2,88.071787,143.79220,83.237149,area_43,ex
2,3,88.093434,143.95190,83.168331,area_43,ex
3,4,88.063898,143.81981,83.317563,area_43,ex
4,5,88.066131,143.97354,83.238470,area_43,ex


In [13]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv(os.path.join(base_folder, 'positions/rank_0_positions.txt'), skiprows=8,
                     delimiter=' ', names=column_headers_pos)
areas = df_pos['area'].unique()

neurons = []
average_Ca = []
Average_pos_per_area = []
brain_area_boundaries = []

df_chosen_pos = pd.DataFrame()

for area in areas:
    df_area = df_pos[df_pos['area'] == area]
    local_neurons = df_area['local id'].unique()
    chosen_neurons = pd.DataFrame(np.random.choice(local_neurons, 5))
    neurons.append(chosen_neurons)
    
    # points = df_pos[df_pos['area'] == area].values  # Extract points (Nx3)
    # print(points[:, 2:4])
    # hull = ConvexHull(points[:, 2:4])  # Only x, y for 2D boundary
    # boundary_points = points[hull.vertices]
    # brain_area_boundaries.append(boundary_points)


column_headers_monitors = ['step', 'calcium']
df_calcium = pd.DataFrame()
Ca_area = pd.DataFrame()
Area_pos = pd.DataFrame()

for neurons_per_area in neurons:
    for neuron in neurons_per_area.to_numpy():
        filename = f'monitors/0_{neuron[0]}.csv'
        temp_df = pd.read_csv(os.path.join(base_folder, filename), skiprows=0, usecols=[
                              0, 5], delimiter=';', names=column_headers_monitors)
        # Add the neuron ID as a new column
        temp_df['neuron_id'] = neuron*np.ones(len(temp_df['calcium']), dtype=int)
        df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

for ix, i in enumerate(neurons):
    average_Ca = df_calcium[df_calcium['neuron_id'].isin(
        i.to_numpy())].groupby('step')['calcium'].mean()
    Average_pos_per_area_calc = df_pos[df_pos['local id'].isin(i[0])][['pos x', 'pos y', 'pos z']].mean(axis=0)
    Area_pos[areas[ix]] = Average_pos_per_area_calc
    Ca_area[areas[ix]] = average_Ca

In [ ]:
# column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

# df_pos = pd.read_csv('rank_0_positions.txt', skiprows=8,
#                      delimiter=' ', names=column_headers_pos)
# areas = df_pos['area'].unique()

# neurons = []
# average_Ca = []
# Average_pos_per_area = []
# brain_area_boundaries = []

# df_chosen_pos = pd.DataFrame()

# for area in areas:
#     df_area = df_pos[df_pos['area'] == area]
#     local_neurons = df_area['local id'].unique()
#     chosen_neurons = pd.DataFrame(local_neurons)
#     neurons.append(chosen_neurons)
    
#     # points = df_pos[df_pos['area'] == area].values  # Extract points (Nx3)
#     # print(points[:, 2:4])
#     # hull = ConvexHull(points[:, 2:4])  # Only x, y for 2D boundary
#     # boundary_points = points[hull.vertices]
#     # brain_area_boundaries.append(boundary_points)


# column_headers_monitors = ['step', 'calcium']
# df_calcium = pd.DataFrame()
# Ca_area = pd.DataFrame()
# Area_pos = pd.DataFrame()

# for neurons_per_area in neurons:
#     for neuron in neurons_per_area.to_numpy():
#         filepath = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/viz-stimulus/monitors/0_{neuron[0]}.csv'
#         temp_df = pd.read_csv(filepath, skiprows=0, usecols=[
#                               0, 5], delimiter=';', names=column_headers_monitors)
#         # Add the neuron ID as a new column
#         temp_df['neuron_id'] = neuron*np.ones(len(temp_df['calcium']), dtype=int)
#         df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

# for ix, i in enumerate(neurons):
#     average_Ca = df_calcium[df_calcium['neuron_id'].isin(
#         i.to_numpy())].groupby('step')['calcium'].mean()
#     Average_pos_per_area_calc = df_pos[df_pos['local id'].isin(i[0])][['pos x', 'pos y', 'pos z']].mean(axis=0)
#     Area_pos[areas[ix]] = Average_pos_per_area_calc
#     Ca_area[areas[ix]] = average_Ca



KeyboardInterrupt: 

In [ ]:
# # Create a VTK renderer and render window
# renderer = vtk.vtkRenderer()
# render_window = vtk.vtkRenderWindow()
# render_window.AddRenderer(renderer)
# render_window_interactor = vtk.vtkRenderWindowInteractor()
# render_window_interactor.SetRenderWindow(render_window)

# # Loop through each brain area's boundary
# for boundary_points in brain_area_boundaries:
#     # Create a polygon
#     points = vtk.vtkPoints()
#     polygon = vtk.vtkPolygon()
#     polygon.GetPointIds().SetNumberOfIds(len(boundary_points))

#     for i, point in enumerate(boundary_points):
#         points.InsertNextPoint(point[0], point[1], point[2])  # Add boundary point
#         polygon.GetPointIds().SetId(i, i)

#     # Create a cell array to store the polygon
#     polygons = vtk.vtkCellArray()
#     polygons.InsertNextCell(polygon)

#     # Create polydata to store points and polygons
#     polydata = vtk.vtkPolyData()
#     polydata.SetPoints(points)
#     polydata.SetPolys(polygons)

#     # Create a mapper and actor
#     mapper = vtk.vtkPolyDataMapper()
#     mapper.SetInputData(polydata)

#     actor = vtk.vtkActor()
#     actor.SetMapper(mapper)
#     actor.GetProperty().SetColor(1, 0, 0)  # Set to red for visibility
#     actor.GetProperty().SetOpacity(0.5)  # Set transparency if needed

#     # Add actor to the renderer
#     renderer.AddActor(actor)

# # Setup camera and rendering
# renderer.SetBackground(0, 0, 0)  # Black background
# render_window.SetSize(800, 800)
# render_window.Render()
# render_window_interactor.Start()


In [4]:
# Create a VTK points object for the positions
points = vtk.vtkPoints()

# Create a VTK float array for calcium levels
calcium_levels = vtk.vtkFloatArray()
calcium_levels.SetName("Calcium Levels")

# Iterate through the areas
areas = Ca_area.columns
for area in areas:
    pos = Area_pos[area].values.reshape(3, -1)  # pos_x, pos_y, pos_z
    calcium = Ca_area[area].values  # Calcium levels

    for i in range(pos.shape[1]):
        # Add point to the points object
        points.InsertNextPoint(pos[0][i], pos[1][i], pos[2][i])
        
        # Add calcium value
        calcium_levels.InsertNextValue(calcium[i])

# Create a polydata object
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)
polydata.GetPointData().SetScalars(calcium_levels)

# Create a sphere source (glyph shape)
sphere_source = vtk.vtkSphereSource()
sphere_source.SetRadius(1)  # Adjust the radius to make spheres larger

# Create a glyph filter to place spheres at each point
glyph = vtk.vtkGlyph3D()
glyph.SetSourceConnection(sphere_source.GetOutputPort())
glyph.SetInputData(polydata)
glyph.ScalingOff()  # Turn off scaling if you don't want the size to vary
glyph.Update()

# Create a mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(glyph.GetOutputPort())
mapper.SetScalarRange(calcium_levels.GetRange())  # Set range for color mapping

# Create an actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Create a renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

# Create a render window
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

# Create a render window interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Add color bar
scalar_bar = vtk.vtkScalarBarActor()
scalar_bar.SetLookupTable(mapper.GetLookupTable())
scalar_bar.SetTitle("Calcium Levels")
renderer.AddActor2D(scalar_bar)

# Start the visualization
render_window.Render()
interactor.Start()


In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv('rank_0_positions.txt', skiprows= 8, delimiter=' ', names=column_headers_pos)
df_pos.head()


In [ ]:
areas = df_pos['area'].unique()
print(len(areas))

In [ ]:
neurons = []
#df_pos_reduced = pd.DataFrame()

for area in areas:
    df_area = df_pos[df_pos['area'] == area]
    local_neurons = df_area['local id'].unique()
    chosen_neurons = pd.DataFrame(np.random.choice(local_neurons, 5))

    neurons.append(chosen_neurons)
    #df_pos_reduced = pd.concat([df_pos_reduced, chosen_neurons], ignore_index=True)



In [ ]:
#neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=100, replace=False))

In [ ]:
column_headers_monitors = ['step', 'calcium']

df_calcium = pd.DataFrame()

for i in neurons:
    for j in i.to_numpy():
        filepath = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/viz-stimulus/monitors/0_{j[0]}.csv'
        temp_df = pd.read_csv(filepath, skiprows=0, usecols=[0, 5], delimiter=';', names=column_headers_monitors)
        temp_df['neuron_id'] = j*np.ones(len(temp_df['calcium']), dtype=int)  # Add the neuron ID as a new column
        df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

df_calcium.tail()

In [ ]:
Ca_area = pd.DataFrame([], ['Calcium_concentration'])
average_Ca = []

for ix, i in enumerate(neurons):
    average_Ca = df_calcium[df_calcium['neuron_id'].isin(
        i.to_numpy())].groupby('step')['calcium'].mean()

    Ca_area[areas[ix]] = average_Ca

Ca_area.head()


In [ ]:
# Create a vtkPoints object to store neuron positions
points = vtk.vtkPoints()
for _, row in df_pos.iterrows():
    if row['local id'] in neurons:
        points.InsertNextPoint(row['pos x'], row['pos y'], row['pos z'])

# Create a vtkPolyData object to store the points
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)

# Create a mapper and actor for the points
vertex_filter = vtk.vtkVertexGlyphFilter()  # Ensures points are rendered
vertex_filter.SetInputData(polydata)
vertex_filter.Update()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(vertex_filter.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Customize point appearance
actor.GetProperty().SetColor(1.0, 1.0, 1.0)  # White points
actor.GetProperty().SetPointSize(2)         # Set point size

# Set up the renderer, render window, and interactor
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Add the actor to the renderer
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Background color: dark gray

# Start the visualization
render_window.Render()
interactor.Start()

# Neuron calcium concentrations plot using Matplotlib

In [ ]:
# Specify the time step to visualize
time_point = 100000
df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

# Merge position and calcium data
df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Normalize calcium concentration for coloring
global_min = df_calcium['calcium'].min()  # Use global min and max for consistent coloring
global_max = df_calcium['calcium'].max()
calcium_values = df_merged['calcium']
colors = plt.cm.viridis((calcium_values - global_min) / (global_max - global_min))

scatter = ax.scatter(
    df_merged['pos x'], df_merged['pos y'], df_merged['pos z'],
    c=colors, s=50, cmap='viridis', edgecolor='k'
)

cbar = fig.colorbar(scatter, ax=ax, shrink=0.5, aspect=10)
cbar.set_label('Calcium Concentration')

ax.set_title(f'Neuron Calcium Concentrations at Time Step {time_point}')

plt.show()

# Neuron calcium concentrations plot using VTK

In [ ]:
# Specify the time step to visualize
time_point = 10000  # Replace with your desired time step

# Filter data for the specified time point
df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

# Merge position and calcium data
df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

# Normalize calcium concentration for coloring
global_min = df_calcium['calcium'].min()  # Use global range
global_max = df_calcium['calcium'].max()
calcium_values = df_merged['calcium']
normalized_calcium = (calcium_values - global_min) / (global_max - global_min)

# Create a VTK points object to hold neuron positions
points = vtk.vtkPoints()
for _, row in df_merged.iterrows():
    points.InsertNextPoint(row['pos x'], row['pos y'], row['pos z'])

# Create a VTK cell array to represent vertices
vertices = vtk.vtkCellArray()
for i in range(len(df_merged)):
    vertices.InsertNextCell(1)
    vertices.InsertCellPoint(i)

# Create a VTK polydata object to store the points and vertices
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)
polydata.SetVerts(vertices)

# Add calcium concentration as a scalar field for coloring
calcium_array = vtk.vtkFloatArray()
calcium_array.SetName("Calcium")
for value in normalized_calcium:
    calcium_array.InsertNextValue(value)
polydata.GetPointData().SetScalars(calcium_array)

# Create a mapper and actor for visualization
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputData(polydata)
mapper.SetScalarRange(0.0, 1.0)  # Normalized range [0, 1]
mapper.SetColorModeToMapScalars()
mapper.SetScalarModeToUsePointData()
mapper.Update()

actor = vtk.vtkActor()
actor.SetMapper(mapper)

actor.GetProperty().SetPointSize(10)  # Adjust the size as needed

# Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Add the actor to the renderer
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background

# Start the visualization
render_window.Render()
interactor.Start()